## Load PDF documents


In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('attention.pdf')
docs = loader.load()

## Split documents into chunks

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

## Create embeddings and store them in a vector DB (Chroma)


In [3]:
# Create embeddings and store them in a vector DB (Chroma)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings()
db = Chroma.from_documents(documents, embeddings)

/var/folders/sc/ldvyw7gd6tv8q7ry2q_605x00000gn/T/ipykernel_25414/2223805540.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/var/folders/sc/ldvyw7gd6tv8q7ry2q_605x00000gn/T/ipykernel_25414/2223805540.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/Users/amanatanayak/Documents/Python3/LLM/LANGCHAIN_TUTORIAL/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:1

## Load LLM from Hugging Face


In [4]:
from langchain_huggingface import HuggingFaceEndpoint
from dotenv import load_dotenv
import os

load_dotenv()

ACCESS_TOKEN = os.getenv('TOKEN')

llm = HuggingFaceEndpoint(
    repo_id='tiiuae/falcon-7b-instruct',
    huggingfacehub_api_token=ACCESS_TOKEN,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/amanatanayak/.cache/huggingface/token
Login successful


## Prompt and Chain Setup


In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}
""")

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)


## Retrieval 

In [6]:
retriever = db.as_retriever()

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)




In [7]:
# Query the chain
response = retriever_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [8]:
response['answer']

'What is the advantage of using the Scaled Dot-Product Attention over the Multi-Head Attention\n(see Answer)\n(see Answer)'